# Sesiunea 5 – Orchestrator Multi-Agent

Demonstrează un flux simplu cu doi agenți (Cercetător -> Editor) folosind Foundry Local.


### Explicație: Instalarea Dependențelor
Instalează `foundry-local-sdk` și `openai`, necesare pentru accesul la modelul local și completările de chat. Idempotent.


# Scenariu
Implementează un model minimal de orchestrare cu doi agenți:
- **Agentul Cercetător** colectează informații concise și factuale
- **Agentul Editor** rescrie pentru claritate executivă

Demonstrează memoria partajată per agent, transmiterea secvențială a rezultatelor intermediare și o funcție simplă de pipeline. Extensibil pentru mai multe roluri (ex. Critic, Verificator) sau ramuri paralele.

**Variabile de mediu:**
- `FOUNDRY_LOCAL_ALIAS` - Modelul implicit de utilizat (implicit: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Modelul principal al agentului (suprascrie ALIAS)
- `AGENT_MODEL_EDITOR` - Modelul agentului Editor (implicit: modelul principal)

**Referință SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Cum funcționează:**
1. **FoundryLocalManager** pornește automat serviciul Foundry Local
2. Descarcă și încarcă modelul specificat (sau folosește versiunea din cache)
3. Oferă un endpoint compatibil cu OpenAI pentru interacțiune
4. Fiecare agent poate utiliza un model diferit pentru sarcini specializate
5. Logica de retry integrată gestionează eșecurile tranzitorii cu ușurință

**Caracteristici cheie:**
- ✅ Descoperire și inițializare automată a serviciului
- ✅ Gestionarea ciclului de viață al modelului (descărcare, cache, încărcare)
- ✅ Compatibilitate cu SDK OpenAI pentru un API familiar
- ✅ Suport multi-model pentru specializarea agenților
- ✅ Gestionare robustă a erorilor cu logica de retry
- ✅ Inferență locală (fără necesitatea unui API cloud)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Explicație: Importuri de Bază & Tipizare
Introduce dataclasses pentru stocarea mesajelor agentului și indicii de tip pentru claritate. Importă Foundry Local manager + clientul OpenAI pentru acțiunile ulterioare ale agentului.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Explicație: Inițializarea Modelului (Model SDK)
Utilizează Foundry Local Python SDK pentru o gestionare robustă a modelelor:
- **FoundryLocalManager(alias)** - Pornește automat serviciul și încarcă modelul prin alias
- **get_model_info(alias)** - Rezolvă aliasul către un ID concret al modelului
- **manager.endpoint** - Oferă punctul de acces pentru clientul OpenAI
- **manager.api_key** - Oferă cheia API (opțional pentru utilizarea locală)
- Suportă modele separate pentru agenți diferiți (primar vs editor)
- Logică de retry integrată cu backoff exponențial pentru reziliență
- Verificarea conexiunii pentru a asigura că serviciul este pregătit

**Model SDK Cheie:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Gestionarea Ciclică:**
- Managerii sunt stocați global pentru o curățare adecvată
- Fiecare agent poate utiliza un model diferit pentru specializare
- Descoperirea automată a serviciului și gestionarea conexiunii
- Retry grațios cu backoff exponențial în caz de eșecuri

Acest lucru asigură o inițializare corectă înainte de începerea orchestrării agenților.

**Referință:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Explicație: Clasele Agent & Memory
Definește `AgentMsg` ca o intrare simplă pentru memorie și `Agent`, care include:
- **Rolul sistemului** - Personalitatea și instrucțiunile agentului
- **Istoricul mesajelor** - Menține contextul conversației
- **Metoda act()** - Execută acțiuni cu gestionarea corectă a erorilor

Agentul poate utiliza modele diferite (principal vs editor) și menține un context izolat pentru fiecare agent. Acest model permite:
- Persistența memoriei între acțiuni
- Atribuirea flexibilă a modelelor pentru fiecare agent
- Izolarea și recuperarea în caz de erori
- Lanțuri și orchestrare ușoară


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Explicație: Pipeline Orchestrat
Creează doi agenți specializați:
- **Cercetător**: Folosește modelul principal, adună informații factuale
- **Editor**: Poate folosi un model separat (dacă este configurat), rafinează și rescrie

Funcția `pipeline`:
1. Cercetătorul adună informații brute
2. Editorul le rafinează într-un rezultat pregătit pentru execuție
3. Returnează atât rezultatele intermediare, cât și cele finale

Acest model permite:
- Specializarea modelelor (modele diferite pentru roluri diferite)
- Îmbunătățirea calității prin procesare în mai multe etape
- Trasabilitatea transformării informațiilor
- Extindere ușoară către mai mulți agenți sau procesare paralelă


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Explicație: Execuția și Rezultatele Pipeline-ului
Execută pipeline-ul multi-agent pe o întrebare tematică despre conformitate + latență pentru a demonstra:
- Transformarea informației în mai multe etape
- Specializarea și colaborarea agenților
- Îmbunătățirea calității rezultatului prin rafinare
- Trasabilitate (atât rezultatele intermediare, cât și cele finale sunt păstrate)

**Structura Rezultatelor:**
- `question` - Întrebarea originală a utilizatorului
- `research` - Rezultatul brut al cercetării (puncte factuale)
- `final` - Rezumat executiv rafinat
- `models` - Modelele utilizate pentru fiecare etapă

**Idei de Extindere:**
1. Adăugarea unui agent Critic pentru revizuirea calității
2. Implementarea agenților de cercetare paralelă pentru diferite aspecte
3. Adăugarea unui agent Verificator pentru verificarea faptelor
4. Utilizarea unor modele diferite pentru niveluri diferite de complexitate
5. Implementarea buclelor de feedback pentru îmbunătățire iterativă


### Avansat: Configurarea Personalizată a Agentului

Încercați să personalizați comportamentul agentului modificând variabilele de mediu înainte de a rula celula de inițializare:

**Modele Disponibile:**
- Utilizați `foundry model ls` în terminal pentru a vedea toate modelele disponibile
- Exemple: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, etc.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
